In [41]:
# Never forget to firs check the .../robot.txt of every website you wish to scrap/crawl over

In [1]:
# load the libraries you'll need
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import scrapy

In [2]:
# Save your html file with this function that way you don't male repeated requests to a website
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

# Open it back up with this function
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

In [3]:
# This is the prefix of the website for later use
prefix = "http://www.ifar.org/"

In [4]:
url = 'http://www.ifar.org/catalogues_raisonnes.php?alpha=&searchtype=artist&published=1&inPrep=1&artist=&author='
path = 'C:/Users/ramon/OneDrive/INSEAD/auction_proj'

In [10]:
# create a conection with the website
r = requests.get(url)
print(r.content[:100])

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtm'


In [11]:
# assign the content of the website to a new variable using the method .content then save to not make more requests
save_html(r.content, 'auctions_html')

In [7]:
# Load back in the hmtl of the page
html = open_html('auctions_html')

In [8]:
# Doing the column with paths using scrapy

In [9]:
# This is the main page
sel = scrapy.Selector(text = html)

In [10]:
# This is just checking all of the hyperlinks in the table
sel.xpath('//tr/td/a/@href').extract()

['artist_name.php?nameid=1563&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2030&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1991&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2292&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1637&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1849&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1636&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1457&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1210&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2709&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1638&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1866&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1640&published=1&inPrep=1&author=',
 'artist_name.php?nameid

In [12]:
# This is everything we need for the first page
ps = sel.xpath('//tr').xpath('./td')
#second = ps.copy()
ps.extract()

['<td class="top_center_name">Name </td>',
 '<td class="top_center_birth_year">Birth Year </td>',
 '<td class="top_center_birth_place">Birth Place </td>',
 '<td class="top_center_death_year">Death Year </td>',
 '<td class="top_center_death_place">Death Place </td>',
 '<td class="g_center_name"><a href="artist_name.php?nameid=1563&amp;published=1&amp;inPrep=1&amp;author=">Aachen, Hans von<span>click to learn more</span></a></td>',
 '<td class="g_center_birth_year">1552</td>',
 '<td class="g_center_birth_place">Germany</td>',
 '<td class="g_center_death_year">1615</td>',
 '<td class="g_center_death_place">Czech Republic</td>',
 '<td class="g_center_name"><a href="artist_name.php?nameid=1396&amp;published=1&amp;inPrep=1&amp;author=">Aalto, Alvar<span>click to learn more</span></a></td>',
 '<td class="g_center_birth_year">1898</td>',
 '<td class="g_center_birth_place">Finland</td>',
 '<td class="g_center_death_year">1976</td>',
 '<td class="g_center_death_place">Finland</td>',
 '<td class=

In [13]:
# This is the second page
second_url = 'http://www.ifar.org/artist_name.php?nameid=7&published=1&inPrep=1&author='
r_dos = requests.get(second_url)
save_html(r_dos.content, 'auctions_dos')

In [14]:
html_dos = open_html('auctions_dos')

In [ ]:
html_dos

In [16]:
sel_dos = scrapy.Selector(text = html_dos)

In [39]:
ps_dos = sel_dos.css('#box_2_left ::text')

lis_dos = []
for p in ps_dos.extract():
    p
    lis_dos.append(p)
#ps_dos.extract().strip()

In [40]:
new_lis = [s.strip() for s in lis_dos]
new_lis

['',
 'Catalogues in Preparation',
 '',
 '',
 '',
 'Title unknown at this time',
 'click to learn more',
 '',
 'Medium Covered: Paintings',
 '',
 'Author: \r\n                                                            \r\n                                                           \r\n                                Redensek, Jeannette',
 '',
 '',
 'Sponsor: The Josef and Anni Albers Foundation',
 '',
 '',
 '',
 '',
 'Published Catalogues',
 '',
 '',
 '',
 'The Prints of Josef Albers : A Catalogue Raisonné, 1915-1976.',
 'click to learn more',
 '',
 'Danilowitz, Brenda;&nbsp\r\n                                                                            Weber, Nicholas Fox (foreword)',
 '',
 'New York : Hudson Hills Press, in association with the Josef and Anni Albers Foundation, 2001.',
 '',
 '',
 '',
 '',
 '',
 '']

In [6]:
response = scrapy.http.HtmlResponse(url = second_url, body = html_dos)

In [51]:
course_data = sel.css('tr > td.g_center_name > a::attr(href)').extract()
course_data

['artist_name.php?nameid=1563&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2030&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1991&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2292&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1637&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1849&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1636&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1457&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1210&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2709&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1638&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1866&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1640&published=1&inPrep=1&author=',
 'artist_name.php?nameid

In [52]:
prefix + course_data[1]

'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author='

In [53]:
urls = []

for d in course_data:
    var = prefix + d
    urls.append(var)

urls[1:10]


['http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=2030&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1991&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=2292&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1637&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1849&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1636&published=1&inPrep=1&author=']

In [56]:
names = sel.css('tr > td.g_center_name > a::text')
#names.extract()

In [57]:
len(names)

2918

In [67]:
birth_year = sel.css('tr > td.g_center_birth_year::text').extract()
#birth_year

In [68]:
len(birth_year)

2916

In [13]:
from scrapy.crawler import CrawlerProcess

In [11]:


class AASpyder(scrapy.Spider):
    name = 'artsy_spyder'
    
    def start_requests( self ):
        for url in urls:
            yield scrapy.Request( url = url, callback = self.parse )

    def parse( self, response ):
        links = response.css('div.catalogue_title > a::attr(href)').extract()       
        for link in links:
            yield response.follow( url = link, callback = self.parse2 )
            
        filepath = 'aa_links.csv'
        with open( filepath, 'w' ) as f:
            f.writelines( [link + '/n' for link in links] )
#    def parse2( self, response):
#        info = response.css('div.book_title::text)').extract()
#        info_ex = [t for t in info.extract()]
        # Store this in our dictionary
#        aa_dict[ info_ex ] = info_ex
            
#            html_file = 'arts_main.html'
#       with open( html_file, 'wb' ) as fout:
#          fout.write( response.body )

In [14]:
# Initialize the dictionary **outside** of the Spider class
aa_dict = dict()

# Run the Spider
process = CrawlerProcess()
process.crawl(AASpyder)
process.start()



2019-11-05 22:01:05 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-11-05 22:01:05 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Windows-10-10.0.17134-SP0
2019-11-05 22:01:05 [scrapy.crawler] INFO: Overridden settings: {}
2019-11-05 22:01:06 [scrapy.extensions.telnet] INFO: Telnet Password: 9ae5f9521f8437b6
2019-11-05 22:01:06 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2019-11-05 22:01:08 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defa

In [16]:
data = pd.read_csv('aa_links.csv')

In [17]:
data.head()

,artist_book_detail_in_progress.php?id=50171&nameid=1099/nartist_book_detail.php?bookid=1217&nameid=1099/nartist_book_detail.php?bookid=1803&nameid=1099/nartist_book_detail.php?bookid=1893&nameid=1099/nartist_book_detail.php?bookid=1894&nameid=1099/nartist_book_detail.php?bookid=41&nameid=1099/n


In [25]:
def previewCourses( aa_dict, n = 3 ):
    info_ex = list( aa_dict.keys() )
    print( "A preview of DataCamp Courses:")
    print("---------------------------------------\n")
    for t in info_ex[:n]:
        print( "TITLE: %s" % t)
        for i,ct in enumerate(aa_dict[t]):
            print("\tChapter %d: %s" % (i+1,ct) )
        print("")

In [26]:
# Print a preview of courses
previewCourses(aa_dict)

A preview of DataCamp Courses:
---------------------------------------



In [18]:
ro = requests.get('http://www.ifar.org/artist_name.php?nameid=28&published=1&inPrep=1&author=')
save_html(ro.content, 'sample_html')

2019-11-05 22:18:10 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): www.ifar.org:80
2019-11-05 22:18:11 [urllib3.connectionpool] DEBUG: http://www.ifar.org:80 "GET /artist_name.php?nameid=1398&published=1&inPrep=1&author= HTTP/1.1" 200 5622


In [22]:
html_2 = open_html('sample_html')

'http://www.ifar.org/artist_name.php?nameid=1398&published=1&inPrep=1&author='

In [28]:
resp = scrapy.http.HtmlResponse(url = url_2, body = html_2)
links = resp.css('div.catalogue_title ::text').extract_first()
links

'\r\n                                '

In [26]:
resp

<200 http://www.ifar.org/artist_name.php?nameid=1398&published=1&inPrep=1&author=>

In [ ]:
 > a::attr(href)